In [115]:
from unicodedata import numeric
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

In [116]:
# df = pd.read_csv('../data/datascience-cis-2022.csv')
# devices = ['bd:0d:8e:c5', 'bd:24:bb:1e', 'bd:0d:af:0b', 'ee:dd:34:1d', 'ee:dd:39:08', 'ee:dd:39:09']
# small_df = df.loc[df['device_mac'].isin(devices)]
# small_df.to_csv('../data/small_data.csv',index=False)

In [149]:
df = pd.read_csv('../data/small_data.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time','device_mac')

In [154]:
df.index = df.index.round('min')

In [157]:
for col in df.columns:
    if (df[col] == 0).all():
        print(col)
# drop these cols

df = df.loc[:, (df != 0).any(axis=0)]

FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount
ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount


In [158]:
def nonzero_proportion(x):
    return (x!=0).mean()

In [159]:
numeric_cols = list(df.columns)[:-1]
funcs = {'mean':np.mean, 'std':np.std, 'max':np.max, 'min':np.min, 'nonzero_p':nonzero_proportion}

In [160]:
df['time_group'] = df.index.to_period('H').strftime('%d_%H')

<ipython-input-160-1b9ae5dd2c07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_group'] = df.index.to_period('H').strftime('%d_%H')


In [162]:
from tsfresh.feature_extraction import extract_features, EfficientFCParameters

    d1 d2 d3 d4
    ===================
time                packet
time2

In [163]:
df['id'] = df['device_mac']+'_'+df['time_group']

<ipython-input-163-c8108f8bdaef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = df['device_mac']+'_'+df['time_group']


In [181]:
# df.drop(columns=['device_mac','time_group']).reset_index().set_index('id').T.unstack(level=1)
tmp = df.drop(columns=['device_mac','time_group']).reset_index().set_index(['time','id']).stack().reset_index()

tmp = tmp.loc[tmp['id'].isin(['bd:0d:8e:c5_02_00','bd:0d:af:0b_02_00'])]

tmp['row_id'] = tmp['time'].astype(str) + '|' + tmp['level_2']

tmp = tmp.drop(columns=['time','level_2']).pivot(columns='id',values=0,index='row_id').reset_index()

tmp[['time','network']] = tmp.apply(lambda x: x['row_id'].split('|'),result_type='expand',axis=1)

tmp.drop(columns='row_id').set_index(['time','network'])

id                                                                      bd:0d:8e:c5_02_00  \
time                network                                                                 
2022-09-02 00:25:00 FromInternetTCPPort554IPurn:ietf:params:mud:con...                0.0   
                    FromInternetTCPPort554IPurn:ietf:params:mud:con...                0.0   
                    FromInternetTCPPort80IPurn:ietf:params:mud:cont...                0.0   
                    FromInternetTCPPort80IPurn:ietf:params:mud:cont...                0.0   
                    FromInternetUDPPort67IPurn:ietf:params:mud:cont...                0.0   
...                                                                                   ...   
2022-09-02 00:59:00 ToInternetUDPPort1024IPurn:ietf:params:mud:cont...             8658.0   
                    ToLocalUDPPort5353IP224.0.0.251/32ByteCount                       0.0   
                    ToLocalUDPPort5353IP224.0.0.251/32PacketCount                     0.0   
                    ToLocalUDPPort67IP255.255.255.255/32ByteCount                     0.0   
                    ToLocalUDPPort67IP255.255.255.255/32PacketCount                   0.0   

id                                                                      bd:0d:af:0b_02_00  
time                network                                                                
2022-09-02 00:25:00 FromInternetTCPPort554IPurn:ietf:params:mud:con...                0.0  
                    FromInternetTCPPort554IPurn:ietf:params:mud:con...                0.0  
                    FromInternetTCPPort80IPurn:ietf:params:mud:cont...            25880.0  
                    FromInternetTCPPort80IPurn:ietf:params:mud:cont...               91.0  
                    FromInternetUDPPort67IPurn:ietf:params:mud:cont...                0.0  
...                                                                                   ...  
2022-09-02 00:59:00 ToInternetUDPPort1024IPurn:ietf:params:mud:cont...                0.0  
                    ToLocalUDPPort5353IP224.0.0.251/32ByteCount                       0.0  
                    ToLocalUDPPort5353IP224.0.0.251/32PacketCount                     0.0  
                    ToLocalUDPPort67IP255.255.255.255/32ByteCount                     0.0  
                    ToLocalUDPPort67IP255.255.255.255/32PacketCount                   0.0  

[630 rows x 2 columns]

,time,id,level_2,0
0,2022-09-02 00:24:59.417,bd:0d:8e:c5_02_00,FromInternetTCPPort554IPurn:ietf:params:mud:co...,0
1,2022-09-02 00:24:59.417,bd:0d:8e:c5_02_00,FromInternetTCPPort554IPurn:ietf:params:mud:co...,0
2,2022-09-02 00:24:59.417,bd:0d:8e:c5_02_00,FromInternetTCPPort80IPurn:ietf:params:mud:con...,0
3,2022-09-02 00:24:59.417,bd:0d:8e:c5_02_00,FromInternetTCPPort80IPurn:ietf:params:mud:con...,0
4,2022-09-02 00:24:59.417,bd:0d:8e:c5_02_00,FromInternetUDPPort67IPurn:ietf:params:mud:con...,0
...,...,...,...,...
2060455,2022-09-16 00:23:55.617,bd:0d:af:0b_16_00,ToInternetUDPPort1024IPurn:ietf:params:mud:con...,0
2060456,2022-09-16 00:23:55.617,bd:0d:af:0b_16_00,ToLocalUDPPort5353IP224.0.0.251/32ByteCount,0
2060457,2022-09-16 00:23:55.617,bd:0d:af:0b_16_00,ToLocalUDPPort5353IP224.0.0.251/32PacketCount,0
2060458,2022-09-16 00:23:55.617,bd:0d:af:0b_16_00,ToLocalUDPPort67IP255.255.255.255/32ByteCount,0


In [12]:
extracted_features = extract_features(df.reset_index().drop(columns=['time_group']), column_id="device_mac", column_sort="time", default_fc_parameters=EfficientFCParameters())

Feature Extraction: 100%|██████████| 10/10 [12:24<00:00, 74.48s/it]


In [14]:
extracted_features

,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__variance_larger_than_standard_deviation,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__has_duplicate_max,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__has_duplicate_min,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__has_duplicate,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__sum_values,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__abs_energy,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__mean_abs_change,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__mean_change,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__mean_second_derivative_central,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount__median,...,ToLocalUDPPort67IP255.255.255.255/32PacketCount__permutation_entropy__dimension_6__tau_1,ToLocalUDPPort67IP255.255.255.255/32PacketCount__permutation_entropy__dimension_7__tau_1,ToLocalUDPPort67IP255.255.255.255/32PacketCount__query_similarity_count__query_None__threshold_0.0,"ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""min""__threshold_0.98","ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""max""__threshold_0.98","ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""mean""__threshold_0.98","ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""median""__threshold_0.98","ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""25""__threshold_0.98","ToLocalUDPPort67IP255.255.255.255/32PacketCount__matrix_profile__feature_""75""__threshold_0.98",ToLocalUDPPort67IP255.255.255.255/32PacketCount__mean_n_absolute_max__number_of_maxima_7
bd:0d:8e:c5,0.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,...,1.538904,1.826191,NaN,3.345079,15.620899,10.279796,10.376745,9.022172,11.741664,4.000000
bd:0d:af:0b,0.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,...,1.525558,1.811798,NaN,3.337966,14.829003,9.843161,9.943650,8.434771,11.332314,4.000000
bd:24:bb:1e,1.0,0.0,1.0,1.0,3.864689e+09,2.519458e+15,7470.366793,40.695511,0.038601,0.0,...,1.533530,1.820466,NaN,3.333974,13.511479,8.729235,8.972066,7.454993,10.150582,4.000000
ee:dd:34:1d,0.0,1.0,1.0,1.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.0,...,1.528470,1.813855,NaN,3.308665,13.991243,8.928342,9.138197,7.345309,10.500693,4.000000
ee:dd:39:08,1.0,0.0,1.0,1.0,4.628879e+09,1.583968e+15,8488.251795,17.170187,0.023483,324646.0,...,1.538620,1.826422,NaN,3.308665,14.833882,9.165715,9.184894,7.747615,10.633845,4.000000
ee:dd:39:09,1.0,0.0,1.0,1.0,2.729134e+09,1.252369e+15,5162.973273,0.000000,0.000000,0.0,...,1.544935,1.834519,NaN,3.340628,13.911117,9.431658,9.652677,8.072929,10.898680,4.285714


In [26]:
summary = {key: [] for key in funcs.keys()}
summary['info'] = []
summary['target'] = []

In [27]:
for time_slice in tqdm(df['time_group'].unique()):
    for device in (df['device_mac'].unique()):
        tmp = df.loc[(df['time_group']==time_slice)&(df['device_mac']==device)][numeric_cols]
        for col in tmp.columns:
            for k in funcs.keys():
                summary[k].append(funcs[k](tmp[col]))
            summary['info'].append(device+'_'+time_slice)
            summary['target'].append(col)

100%|██████████| 337/337 [00:54<00:00,  6.23it/s]


In [28]:
summary = pd.DataFrame(summary)
summary.to_csv('../data/transformed_data.csv',index=False)

In [29]:
target_mapper = {v:i for i,v in enumerate(sorted(summary['target'].unique()))}

In [30]:
import json

In [31]:
with open('target_mappings.json','w') as f:
    json.dump(target_mapper, f)